In [1]:
# Importing required modules and helper functions for SVM modeling
import sys

sys.path.append(r'e:\Data science\Titanic dataset\notebooks')
# Import needed libraries
from auto_imports import *
model_results_imports()
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder , RobustScaler , StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split , GridSearchCV

### Data Loading

Load the processed training and prediction datasets for SVM modeling.

In [2]:
# Load the processed data for training and prediction
df = pd.read_csv(r'E:\Data science\Titanic dataset\data\Processed data\Data Analysis\processed_data.csv')
train_df = df[df['ind'] == 'train'].drop('ind' , axis = 1)
predict_df = df[df['ind'] == 'test'].drop('ind' , axis =1)

train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cells_count,Cabin_letter,Title,FamilySize,Is Alone,Is Special Agent/route,TicketBatch,SharedTicket,Fare per Person,Fare per Age,Tickets counts
0,1,0.0,3,male,22.0,1,0,7.2500,S,3,C,Mr,2,0,1,0,0,3.62500,0.329545,1
1,2,1.0,1,female,38.0,1,0,71.2833,C,1,C,Mrs,2,0,1,0,0,35.64165,1.875876,2
2,3,1.0,3,female,26.0,0,0,7.9250,S,3,C,Miss,1,1,1,0,0,7.92500,0.304808,1
3,4,1.0,1,female,35.0,1,0,53.1000,S,1,C,Mrs,2,0,0,0,0,26.55000,1.517143,2
4,5,0.0,3,male,35.0,0,0,8.0500,S,3,C,Mr,1,1,0,0,0,8.05000,0.230000,1


In [3]:
# Display the first few rows of the training data for inspection
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cells_count,Cabin_letter,Title,FamilySize,Is Alone,Is Special Agent/route,TicketBatch,SharedTicket,Fare per Person,Fare per Age,Tickets counts
0,1,0.0,3,male,22.0,1,0,7.2500,S,3,C,Mr,2,0,1,0,0,3.62500,0.329545,1
1,2,1.0,1,female,38.0,1,0,71.2833,C,1,C,Mrs,2,0,1,0,0,35.64165,1.875876,2
2,3,1.0,3,female,26.0,0,0,7.9250,S,3,C,Miss,1,1,1,0,0,7.92500,0.304808,1
3,4,1.0,1,female,35.0,1,0,53.1000,S,1,C,Mrs,2,0,0,0,0,26.55000,1.517143,2
4,5,0.0,3,male,35.0,0,0,8.0500,S,3,C,Mr,1,1,0,0,0,8.05000,0.230000,1


In [ ]:
# Split features and target variable
X = train_RF_df.drop(['Survived' , 'PassengerId'] , axis =1)
y= train_RF_df['Survived']

### Model Definition

Define and configure the Support Vector Machine classifier.

In [5]:
# Split the data into training and testing sets for model evaluation
X_train , X_test , y_train , y_test = train_test_split(
    X , y , test_size=0.2 , random_state=18
)

In [6]:
# restore preprocessor variable of the columns transformer

%store -r preprocessor

In [7]:
# making the model pipeline

pipeline = Pipeline(
    steps=[
        ('perprocessor',preprocessor),
        ('svm' , SVC(random_state=42 , class_weight='balanced'))
        
    ]
)

In [ ]:
# Hyperparameters tuning 
from sklearn.metrics import accuracy_score , f1_score , make_scorer , precision_score , recall_score
# scoring dict 
scoring = {
    'accuracy' : make_scorer(accuracy_score),
    'f1_score' : make_scorer(f1_score),
    'precision': make_scorer(precision_score , pos_label = 1),
    'recall' : make_scorer(recall_score, pos_label=1)
}
    # hyperparameters
param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'svm__degree': [2, 3, 4],  # Only used for 'poly' kernel
    'svm__gamma': ['scale', 'auto', 0.01, 0.1, 1],  # Used for 'rbf', 'poly', 'sigmoid'
    'svm__coef0': [0.0, 0.1, 0.5],  # Used for 'poly', 'sigmoid'
    'svm__shrinking': [True, False]
}


# grid search algorithm to find best hyperparameter
grid_search = GridSearchCV(pipeline ,
                           param_grid ,
                           cv=5 ,
                           scoring=scoring ,
                           refit='f1_score' , 
                           n_jobs=-1 , 
                           verbose=1)

grid_search.fit(X_train , y_train) # fit the model

Fitting 5 folds for each of 1440 candidates, totalling 7200 fits


In [ ]:
print(f'Best hyperparameters found: {grid_search.best_params_}')
print(f'\nBest cross valdiation f1 socres: {grid_search.best_score_:.4f}')

In [ ]:
svm = grid_search.best_estimator_

In [ ]:
# Evaluate model using cross-validation for accuracy and F1 score
accuracy , f1 = accuracy_f1_scores(X_train , y_train , svm)

In [ ]:
# Evaluate model performance and check for overfitting/underfitting
accuracy_DP , f1_DP , y_pred = drop_percentages(X_test , y_test , svm , accuracy , f1).values()

print('Accuracy drop percentage: ' , accuracy_DP)
print('F1 drop percentage: ' , f1_DP)

In [ ]:
# Show classification report for predictions on the test set
print(classification_report(y_test ,y_pred))

In [ ]:
# Plot precision-recall curve to help select an optimal threshold
from sklearn.metrics import precision_recall_curve 

y_probs = svm.predict_proba(X_test)[:, 1]  # get probability of class 1

# calculate precision, recall and thresholds
precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
idx = np.argmin(np.abs(precision - recall))
precisoin_at_intersection = precision[idx]
recall_at_intersection = recall[idx]
threshold_at_intersection = thresholds[idx]
print('intersection precision point: ',precisoin_at_intersection)
print('intersection recall point: ' , recall_at_intersection)
print('intersection threshold point: ' , threshold_at_intersection)
plt.plot(thresholds, precision[:-1], label='Precision')
plt.plot(thresholds, recall[:-1], label="Recall")

plt.xlabel("Threshold")
plt.ylabel("Score")
plt.title("Precision-Recall vs Threshold")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Apply custom threshold for final predictions and show classification report
y_probs = svm.predict_proba(X_test)[:, 1]  # get probability of class 1

#
y_pred_custom = y_probs >= 0.59

print(classification_report(y_test , y_pred_custom))

In [ ]:
# Evaluate model using cross-validation for accuracy and F1 score
accuracy , f1 = accuracy_f1_scores(X_train , y_train , svm) # getting the results

In [ ]:
# Plot confusion matrix for the custom threshold predictions
cm = confusion_matrix(y_test , y_pred_custom) 
plt.title('Y truth vs Y predicted')
ax = sns.heatmap(cm ,annot=True)
plt.xlabel('Y predicted')
plt.ylabel('Y truth')
ax.set_xticklabels(['Not survived' , 'Survived'])
ax.set_yticklabels(['Not survived' , 'Survived'])

In [ ]:
# Save model predictions for the prediction set for submission or ensembling
save_model_predictions(svm.predict(predict_df.drop('PassengerId' , axis=1)) , 'SupportVectorMachine')

In [ ]:
# Save the trained SVM model for later use (e.g., in ensembling)
import joblib

joblib.dump(svm ,'E:\Data science\Titanic dataset\Models\SupportVectorMachine.pkl')